In [1]:
import os
import pickle
import time
from datetime import datetime
from gls_gen import GLS_GEN

GRAPH_FILENAME = "Graph500.txt"
MAX_ITERATIONS = 1_000 # 256 
RANDOM_SEED_BASE = 4288 # seed for the seed, will be modified for each run
NUM_RUNS = 3
POP_SIZES = [ 100,] 
OUTPUT_DIR = './pckl/GGLS/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Graph: {GRAPH_FILENAME}")
print(f"Max Iterations: {MAX_ITERATIONS}")
print(f"Population Sizes: {POP_SIZES}")
print(f"Number of Runs per Pop Size: {NUM_RUNS}")
print(f"Base Random Seed: {RANDOM_SEED_BASE}")
print(f"Output Directory: {OUTPUT_DIR}")
print("-" * 42) # Your separator length

# Loop through each population size
for pop_size in POP_SIZES:
    print(f"--- Processing Population Size = {pop_size} ---")
    results_for_this_pop_size = []

    for run_num in range(1, NUM_RUNS + 1):
        # Generate a unique seed
        current_seed = RANDOM_SEED_BASE + pop_size + run_num

        print(f"  Running Run {run_num}/{NUM_RUNS} (Seed: {current_seed})...")

        
        gls_runner = GLS_GEN(
            graph_filename=GRAPH_FILENAME,
            pop_size=pop_size,
            max_iterations=MAX_ITERATIONS,
            random_seed=current_seed
        )

        # Run 
        run_start_time = time.time()
        best_cut_found = gls_runner.run_gls()
        run_end_time = time.time()
        print(f"    Completed in {run_end_time - run_start_time:.2f}s. Best Cut: {best_cut_found}")

        # Get  statistics
        run_stats = gls_runner.get_run_statistics()

        # Add run data  
        run_stats['run_number'] = run_num
        run_stats['run_seed'] = current_seed
        # Add a timestamp for this specific 
        run_stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        results_for_this_pop_size.append(run_stats)


    consolidated_timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

    
    output_filename = f"gls_pop{pop_size}_iter_count_{MAX_ITERATIONS}_runs{NUM_RUNS}_{consolidated_timestamp_str}.pkl"
    output_filepath = os.path.join(OUTPUT_DIR, output_filename)

    
    with open(output_filepath, 'wb') as f_out:
        pickle.dump(results_for_this_pop_size, f_out) 

    print(f"--- Finished all {NUM_RUNS} runs for Population Size = {pop_size} ---")
    print("-" * 42) 




Graph: Graph500.txt
Max Iterations: 1000
Population Sizes: [100]
Number of Runs per Pop Size: 3
Base Random Seed: 4288
Output Directory: ./pckl/GGLS/
------------------------------------------
--- Processing Population Size = 100 ---
  Running Run 1/3 (Seed: 4389)...
    Completed in 4845.09s. Best Cut: 5
  Running Run 2/3 (Seed: 4390)...
    Completed in 4462.77s. Best Cut: 2
  Running Run 3/3 (Seed: 4391)...
    Completed in 3966.37s. Best Cut: 8
--- Finished all 3 runs for Population Size = 100 ---
------------------------------------------


In [3]:
import os
import pickle
from collections import defaultdict
from datetime import datetime

# --- Configuration ---
# Directory where the original pickle files are located,
# relative to where you run the script.
INPUT_DIR = './pckl/GGLS/' 

# Directory where the merged pickle files will be saved,
# relative to where you run the script.
OUTPUT_DIR_MERGED = './pckl/merged/' 

# Create the output directory if it doesn't exist
# This will correctly create w:\...\Evo_Comp_Assignment_2\pckl\merged\
os.makedirs(OUTPUT_DIR_MERGED, exist_ok=True)

# Iteration count is constant in your example
ITERATION_COUNT = 1000 

# Population sizes identified from your file list
POP_SIZES_TO_PROCESS = [100, 10, 20, 50, 6] 

# --- Logic ---

print(f"Scanning directory: {os.path.abspath(INPUT_DIR)}") # Show absolute path being scanned
print(f"Looking for Pop Sizes: {POP_SIZES_TO_PROCESS} with Iter Count: {ITERATION_COUNT}")
print(f"Merged files will be saved to: {os.path.abspath(OUTPUT_DIR_MERGED)}") # Show absolute path for output
print("-" * 40)

# Check if INPUT_DIR exists before trying to list its contents
if not os.path.isdir(INPUT_DIR):
    print(f"Error: Input directory not found at {os.path.abspath(INPUT_DIR)}")
    print("Please ensure the script is run from the correct parent directory ('Evo_Comp_Assignment_2')")
    exit() # Stop the script if the input directory isn't found

all_files_in_dir = [f for f in os.listdir(INPUT_DIR) if f.endswith(".pkl")]

if not all_files_in_dir:
    print(f"Warning: No .pkl files found in {os.path.abspath(INPUT_DIR)}")

# Group files manually based on known pop sizes
grouped_files = defaultdict(list)

for pop_size in POP_SIZES_TO_PROCESS:
    # Construct the expected prefix for filenames of this group
    file_prefix = f"gls_pop{pop_size}_iter_count_{ITERATION_COUNT}_"
    
    for filename in all_files_in_dir:
        if filename.startswith(file_prefix):
            # Store the full path relative to the script's execution directory
            grouped_files[pop_size].append(os.path.join(INPUT_DIR, filename)) 
            print(f"  Found for Pop {pop_size}: {filename}") # Print just the filename for clarity

print("\n--- Grouping Complete ---")
for pop_size, files in grouped_files.items():
    if files:
        print(f"Group Pop {pop_size}: {len(files)} file(s)")
        # for f in files:
        #    print(f"    - {f}") # Print full path if needed for debugging
    else:
        print(f"Group Pop {pop_size}: No files found matching prefix.")


print("\n--- Merging Files ---")

# Iterate through each group and merge the data
for pop_size, filepaths in grouped_files.items(): # Note: Renamed 'filenames' to 'filepaths' for clarity
    if len(filepaths) < 1: 
        print(f"Skipping group Pop {pop_size}: No files listed.")
        continue
        
    print(f"Processing group: Population Size = {pop_size}, Iterations = {ITERATION_COUNT}")
    
    combined_results = []
    total_runs_loaded = 0

    # Now iterate through the full filepaths
    for filepath in filepaths: 
        # Extract just the filename for printing, if desired
        filename_only = os.path.basename(filepath) 
        print(f"  Loading data from: {filename_only}") 
        try:
            # Open using the full filepath
            with open(filepath, 'rb') as f_in: 
                data = pickle.load(f_in)
                if isinstance(data, list):
                    combined_results.extend(data)
                    total_runs_loaded += len(data) 
                    print(f"    -> Loaded {len(data)} run results.")
                else:
                    print(f"    -> Warning: Expected a list but got {type(data)}. Skipping file content.")
        except FileNotFoundError:
            # This shouldn't happen if os.listdir worked, but good to keep
            print(f"    -> Error: File not found: {filepath}") 
        except pickle.UnpicklingError as e:
            print(f"    -> Error: Could not unpickle file {filepath}: {e}")
        except Exception as e:
            print(f"    -> Error: An unexpected error occurred loading {filepath}: {e}")

    if not combined_results:
        print(f"  No data successfully loaded for group Pop {pop_size}. Skipping merge.")
        continue

    # Save the combined data to a new pickle file in the correct output directory
    merged_timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"gls_pop{pop_size}_iter_count_{ITERATION_COUNT}_runs{total_runs_loaded}_merged_{merged_timestamp_str}.pkl"
    # Construct the full output path
    output_filepath = os.path.join(OUTPUT_DIR_MERGED, output_filename) 

    print(f"  Saving combined data ({total_runs_loaded} total runs) to: {output_filename} (in {OUTPUT_DIR_MERGED})")
    try:
        with open(output_filepath, 'wb') as f_out:
            pickle.dump(combined_results, f_out)
        print(f"  Successfully saved merged file.")
    except Exception as e:
        print(f"  Error: Could not save merged file {output_filepath}: {e}")
        
    print("-" * 20) 

print("\n--- Merging Process Finished ---")
print(f"Merged files should be in: {os.path.abspath(OUTPUT_DIR_MERGED)}")

Scanning directory: w:\Google_drive\Sync\UU\1_3\Evo\HW\2\Evo_Comp_Assignment_2\pckl\GGLS
Looking for Pop Sizes: [100, 10, 20, 50, 6] with Iter Count: 1000
Merged files will be saved to: w:\Google_drive\Sync\UU\1_3\Evo\HW\2\Evo_Comp_Assignment_2\pckl\merged
----------------------------------------
  Found for Pop 100: gls_pop100_iter_count_1000_runs3_20250331_225146.pkl
  Found for Pop 100: gls_pop100_iter_count_1000_runs3_20250401_160110.pkl
  Found for Pop 100: gls_pop100_iter_count_1000_runs4_20250401_170402.pkl
  Found for Pop 10: gls_pop10_iter_count_1000_runs3_20250331_170234.pkl
  Found for Pop 10: gls_pop10_iter_count_1000_runs7_20250331_191112.pkl
  Found for Pop 20: gls_pop20_iter_count_1000_runs3_20250331_174635.pkl
  Found for Pop 20: gls_pop20_iter_count_1000_runs7_20250331_204756.pkl
  Found for Pop 50: gls_pop50_iter_count_1000_runs3_20250331_193116.pkl
  Found for Pop 50: gls_pop50_iter_count_1000_runs7_20250401_003758.pkl
  Found for Pop 6: gls_pop6_iter_count_1000_runs